In [2]:
from time import time
import sys
board = [
    [0,6,0,1,0,4,0,5,0],
    [0,0,8,3,0,5,6,0,0],
    [2,0,0,0,0,0,0,0,1],
    [8,0,0,4,0,7,0,0,6],
    [0,0,6,0,0,0,3,0,0],
    [7,0,0,9,0,1,0,0,4],
    [5,0,0,0,0,0,0,0,2],
    [0,0,7,2,0,6,9,0,0],
    [0,4,0,5,0,8,0,7,0]
]


def print_board(board):
    for i in range(len(board)):
        if i % 3 == 0 and i != 0:
            print("-----------------------")
        for j in range(len(board[0])):
            if j % 3 == 0 and j != 0:
                print(" | ", end="")
            print(str(board[i][j]) + " ", end="")
            if j == 8:
                print()
    
def find_empty(board):
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:
                return (i, j) 
    return None  

def is_valid(board, num, pos):
    row, col = pos
    if num in board[row]:
        return False
    if num in [board[i][col] for i in range(len(board))]:
        return False
    box_x, box_y = col // 3, row // 3
    for i in range(box_y * 3, box_y * 3 + 3):
        for j in range(box_x * 3, box_x * 3 + 3):
            if board[i][j] == num:
                return False
    return True  

def find_empty_with_mrv(board):
    min_options = 10  
    mrv_cell = None  
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:  
                options = 0
                for num in range(1, 10):
                    if is_valid(board, num, (i, j)):
                        options += 1
                if options < min_options:
                    min_options = options
                    mrv_cell = (i, j)
    return mrv_cell


def find_empty_with_degree(board):
    max_constraints = -1
    degree_cell = None
    for i in range(len(board)):
        for j in range(len(board[0])):
            if board[i][j] == 0:  
                constraints = sum(1 for x in board[i] if x == 0) + sum(1 for row in board if row[j] == 0)
                constraints -= 1
                box_x, box_y = j // 3, i // 3
                for x in range(box_y * 3, box_y * 3 + 3):
                    for y in range(box_x * 3, box_x * 3 + 3):
                        if board[x][y] == 0:
                            constraints += 1
                constraints -= (constraints - max_constraints) // 9 
                if constraints > max_constraints:
                    max_constraints = constraints
                    degree_cell = (i, j)
    return degree_cell


def least_constraining_value(board, pos):
    row, col = pos
    options = []
    for num in range(1, 10):
        if is_valid(board, num, (row, col)):
            constraints = 0
            # Check row and column
            for i in range(len(board)):
                if board[row][i] == 0 and is_valid(board, num, (row, i)):
                    constraints += 1
                if board[i][col] == 0 and is_valid(board, num, (i, col)):
                    constraints += 1
            # Check box
            box_x, box_y = col // 3, row // 3
            for i in range(box_y * 3, box_y * 3 + 3):
                for j in range(box_x * 3, box_x * 3 + 3):
                    if board[i][j] == 0 and is_valid(board, num, (i, j)):
                        constraints += 1
            options.append((num, constraints))
    options.sort(key=lambda x: -x[1])
    return [num for num, _ in options]

def solve(board, choice):
    find = None
    if choice == "1":
        find = find_empty_with_mrv(board)
    elif choice == "2":  
        find = find_empty_with_degree(board)
    elif choice == "3":
        find = find_empty(board) 
    else:
        print("Invalid choice")
        return False
    
    if not find:
        return True
    row, col = find

    numbers_to_try = least_constraining_value(board, (row, col)) if choice == "LCV" else range(1, 10)
    for num in numbers_to_try:
        if is_valid(board, num, (row, col)):
            board[row][col] = num
            if solve(board, choice):
                return True
            board[row][col] = 0

    return False


def enforce_arc_consistency(board):
    queue = [(r, c) for r in range(9) for c in range(9) if board[r][c] == 0]
    while queue:
        row, col = queue.pop(0)
        for value in range(1, 10):
            if not is_valid(board, value, (row, col)):
                continue
            board[row][col] = value
            for i in range(9):
                if i != col and not is_valid(board, board[row][i], (row, i)):
                    board[row][col] = 0
                    break
                if i != row and not is_valid(board, board[i][col], (i, col)):
                    board[row][col] = 0
                    break
            else:
                continue
            break
        else:
            continue
        return False  # If no valid assignment exists
    return True



def preprocess_with_ac3(board):
    domains = enforce_arc_consistency(board)
    if not domains:  
        return False
    new_board = [[0 for _ in range(9)] for _ in range(9)]
    for (row, col), values in domains.items():
        if len(values) == 1:
            new_board[row][col] = next(iter(values))
    return new_board


def solve_with_ac3(board, choice):
    new_board = preprocess_with_ac3(board)
    if new_board is False:
        print("No solution exists according to AC-3.")
        return False
    print("Board after AC-3 preprocessing:")
    print_board(new_board)
    return solve(new_board, choice)




def solve(board, choice):
    find = None
    if choice == "1":
        find = find_empty_with_mrv(board)
    elif choice == "2":  
        find = find_empty_with_degree(board)
    elif choice == "3":  
        find = find_empty(board)        
    else:
        print("Invalid choice")
        return False

    if not find:
        return True

    row, col = find

    if choice == "3":
        numbers_to_try = least_constraining_value(board, (row, col))
    else:
        numbers_to_try = range(1, 10)

    for num in numbers_to_try:
        if is_valid(board, num, (row, col)):
            board[row][col] = num
            if solve(board, choice):
                return True
            board[row][col] = 0

    return False


start = time()        
print_board(board)
#choice = input("\nChoose Heuristic(1,2,3)")
solve(board, '1')
#solve_with_ac3(board, '1')
print("\n")
print_board(board)
end = time()
print("Execution time:", end - start, "seconds")


0 6 0  | 1 0 4  | 0 5 0 
0 0 8  | 3 0 5  | 6 0 0 
2 0 0  | 0 0 0  | 0 0 1 
-----------------------
8 0 0  | 4 0 7  | 0 0 6 
0 0 6  | 0 0 0  | 3 0 0 
7 0 0  | 9 0 1  | 0 0 4 
-----------------------
5 0 0  | 0 0 0  | 0 0 2 
0 0 7  | 2 0 6  | 9 0 0 
0 4 0  | 5 0 8  | 0 7 0 


9 6 3  | 1 7 4  | 2 5 8 
1 7 8  | 3 2 5  | 6 4 9 
2 5 4  | 6 8 9  | 7 3 1 
-----------------------
8 2 1  | 4 3 7  | 5 9 6 
4 9 6  | 8 5 2  | 3 1 7 
7 3 5  | 9 6 1  | 8 2 4 
-----------------------
5 8 9  | 7 1 3  | 4 6 2 
3 1 7  | 2 4 6  | 9 8 5 
6 4 2  | 5 9 8  | 1 7 3 
Execution time: 0.01800084114074707 seconds


In [3]:
import numpy as np
import random

def fitness(square):
    #calculates how close it is the the actual result
    magic_number = 15
    cost = 0

    #calculating the sum of rows and cols
    for i in range(3):
        cost += abs(magic_number - sum(square[i, :]))  #rows
        cost += abs(magic_number - sum(square[:, i]))  #cols
        
    #calculate sum of diagonals
    cost += abs(magic_number - sum([square[i, i] for i in range(3)]))  
    cost += abs(magic_number - sum([square[i, 2-i] for i in range(3)]))  

    return cost  

def swap_elements(square):
    #randomly swap two elements but dont swap central 9
    idx1 = tuple(random.sample(range(3), 2))
    idx2 = tuple(random.sample(range(3), 2))
    while square[idx1] == 9 or square[idx2] == 9:  
        idx1 = tuple(random.sample(range(3), 2))
        idx2 = tuple(random.sample(range(3), 2))
    square[idx1], square[idx2] = square[idx2], square[idx1]
    return square

# random fill and shuffle
square = np.arange(1, 10).reshape((3, 3))
np.random.shuffle(square.flat)

best_fitness = fitness(square)
generations = 0
max_generations = 10000
stale_generations = 0
best_square = np.copy(square)

#evolutionary algorthim
while best_fitness > 0 and generations < max_generations and stale_generations < 1000:
    #generate a new candidate by copying and modifying the current best square
    candidate_square = np.copy(best_square)
    candidate_square = swap_elements(candidate_square)
    current_fitness = fitness(candidate_square)

    #if new cnadidate is better than then update best 
    if current_fitness < best_fitness:
        best_fitness = current_fitness
        best_square = np.copy(candidate_square)
        stale_generations = 0  
    else:
        stale_generations += 1
    generations += 1

# Return the best square found
best_square


C:\Users\muham\AppData\Local\Temp\ipykernel_14844\2053573376.py:32: UserWarning: you are shuffling a 'flatiter' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(square.flat)


array([[7, 1, 6],
       [4, 3, 9],
       [5, 8, 2]])